# 데이터 입력

# 01 데이터 전처리
#### 1) 주문내역 raw 테이블에서 ①배달소요시간을 계산하고, ②10분 단위로 평균 배달소요시간을 계산하는 쿼리
#### 2) 주문내역 테이블과 stat테이블을 조인하는 쿼리

In [26]:
ord_raw.head(5)

,OrderCreatedAt_10m,AvgDeliveryTime
0,2024-12-10 00:00:00,6.286239
1,2024-12-10 00:01:00,7.726161
2,2024-12-10 00:02:00,4.953412
3,2024-12-10 00:03:00,9.456443
4,2024-12-10 00:04:00,6.220976


In [25]:
ord_rider_stat.head(5)

,EventAt,order_cnt,rider_cnt
0,2024-12-10 08:00:00,88,55
1,2024-12-10 08:10:00,81,57
2,2024-12-10 08:20:00,95,68
3,2024-12-10 08:30:00,106,65
4,2024-12-10 08:40:00,118,68


In [14]:
# ① 주문내역 raw 테이블에서 1) 배달소요시간을 계산하고, 2) 10분 단위로 평균 배달소요시간을 계산하는 쿼리 

import sqlite3
import pandas as pd

## SQLite 메모리 데이터베이스 연결
conn = sqlite3.connect(':memory:')

## DataFrame을 SQLite 테이블로 변환 (ord_rider_stat 데이터프레임 사용)
ord_raw.to_sql('ord_raw', conn, index=False, if_exists='replace')

## SQL 쿼리 정의 (SQLite에 맞게 변환)
query = """
    SELECT 
        strftime('%Y-%m-%d %H:%M:00', OrderCreatedAt) AS OrderCreatedAt_10m, 
        AVG((julianday(DeliveryCompletedAt) - julianday(OrderCreatedAt)) * 24 * 60) AS AvgDeliveryTime 
    FROM ord_raw
    GROUP BY 1
    ORDER BY 1
"""

## SQL 쿼리 실행하여 결과 가져오기
ord_raw = pd.read_sql(query, conn)

## 연결 종료
conn.close()

In [16]:
ord_rider_stat.head(5)

,EventAt,order_cnt,rider_cnt
0,2024-12-10 08:00:00,88,55
1,2024-12-10 08:10:00,81,57
2,2024-12-10 08:20:00,95,68
3,2024-12-10 08:30:00,106,65
4,2024-12-10 08:40:00,118,68


In [17]:
# 주문내역 테이블과 stat테이블을 조인하는 쿼리

import sqlite3
import pandas as pd

## SQLite 메모리 데이터베이스 연결
conn = sqlite3.connect(':memory:')

## DataFrame을 SQLite 테이블로 변환 (ord_rider_stat 데이터프레임 사용)
ord_raw.to_sql('ord_raw', conn, index=False, if_exists='replace')
ord_rider_stat.to_sql('ord_rider_stat', conn, index=False, if_exists='replace')

## SQL 쿼리 정의 (SQLite에 맞게 변환)
query = """
    SELECT t1.*
        , AvgDeliveryTime
    FROM ord_rider_stat AS t1     
    INNER JOIN ord_raw AS t2
           ON t1.EventAt = t2.OrderCreatedAt_10m
    GROUP BY 1
    ORDER BY 1
"""

## SQL 쿼리 실행하여 결과 가져오기
final = pd.read_sql(query, conn)

## 연결 종료
conn.close()

In [18]:
final.head(30)

,EventAt,order_cnt,rider_cnt,AvgDeliveryTime
0,2024-12-10 08:00:00,88,55,4.917575
1,2024-12-10 08:10:00,81,57,1.634690
2,2024-12-10 08:20:00,95,68,7.956968
3,2024-12-10 08:30:00,106,65,7.097087
4,2024-12-10 08:40:00,118,68,6.759795
5,2024-12-10 08:50:00,135,66,13.401367
6,2024-12-10 09:00:00,145,68,2.451661
7,2024-12-10 09:10:00,145,95,8.319440
8,2024-12-10 09:20:00,156,83,11.066379
9,2024-12-10 09:30:00,179,102,9.297086


# 02 파라미터 찾기
### 배달품질과 가장 상관도가 높은 최적의 파라미터값 찾기

In [19]:
# 분석 환경 세팅
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


# MAPE (평균 절대 백분율 오차) 구하는 함수 정의
def MAPE(y, pred):
    return np.mean(np.abs((y - pred) / y)) * 100

In [27]:
import decimal
import numpy as np
from tqdm import tqdm

# float형으로 mylist1을 생성
mylist = np.arange(0.1, 1.0, 0.1)

print(mylist)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [29]:
# 진행 상황 표시줄(progress bar)을 쉽게 구현할 수 있게 해주는 라이브러리
from tqdm import tqdm

# 계수값을 담는 리스트를 생성
coef_list = []

In [30]:
# mylist1의 모든 조합을 반복

for w1 in tqdm(mylist):
    for w2 in tqdm(mylist):
                                                         
        # 주문/라이더 관제지표식 계산

        order_parameter = (final['order1_cnt'] * w1 + final['order1_cnt'] * w2 + final['order1_cnt'] * w3 + final['order1_cnt'] * w4 
        rider_parameter = final['rider_cnt'] * 100 * w2
                    
        # ratio 계산 
        final['ratio'] = order_parameter / rider_parameter
        final = final.replace([np.inf, -np.inf], np.nan).dropna(subset=['ratio', 'AvgDeliveryTime']) # 결측치(NaN) 및 무한대값 제거

        # 상관계수 계산
        corr = final[['AvgDeliveryTime', 'ratio']].iloc[0, 1]


        # 회귀 모델 학습 및 예측

        y_col = ["AvgDeliveryTime"]
        x_cols = ["ratio"]
        X = final[x_cols].values
        y = final[y_col].values

        reg = LinearRegression().fit(X, y)
        y_pred = reg.predict(X)

        # RMSE 계산
        #rmse = mean_absolute_error(y, y_pred) * 0.5

        # 결과 저장
        coef_list.append([w1, w2, corr])

100%|██████████| 9/9 [00:00<00:00, 390.95it/s]

100%|██████████| 9/9 [00:00<00:00, 406.66it/s]

100%|██████████| 9/9 [00:00<00:00, 408.96it/s]

100%|██████████| 9/9 [00:00<00:00, 416.77it/s]

100%|██████████| 9/9 [00:00<00:00, 428.08it/s]

100%|██████████| 9/9 [00:00<00:00, 419.82it/s]

100%|██████████| 9/9 [00:00<00:00, 418.91it/s]

100%|██████████| 9/9 [00:00<00:00, 42.64it/s]


In [23]:
coef_list = pd.DataFrame(coef_list)
coef_list.columns = ['w1','w2','corr']

coef_list.sort_values(by=['corr'], ascending = False).head(20)

,w1,w2,corr
90,4.6,0.1,0.736000
80,4.1,0.1,0.656000
70,3.6,0.1,0.576000
60,3.1,0.1,0.496000
50,2.6,0.1,0.416000
40,2.1,0.1,0.336000
30,1.6,0.1,0.256000
20,1.1,0.1,0.176000
91,4.6,0.6,0.122667
81,4.1,0.6,0.109333
